In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from wordcloud import STOPWORDS
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')#Sentence tokenizer
import pickle
from sklearn.model_selection import train_test_split
import zipfile
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Joaquin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
dir_data_train = './train.csv'
dir_data_test = './test.csv'
df_test = pd.read_csv(dir_data_test)
df_train = pd.read_csv(dir_data_train)

In [5]:
df_train.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df_train.keyword = df_train.keyword.apply(lambda x: str(x))
df_train.keyword = df_train.keyword.apply(lambda x:'no_keyword' if x == 'nan' else x)

In [7]:
#Clean text from noise
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text


In [8]:
df_train['clean_text'] = df_train.text.apply(lambda x: clean_text(x))

In [9]:
df_train['X'] = df_train['clean_text'] + ' ' + df_train['keyword']

In [11]:
df_train.X

0       our deeds are the reason of this  earthquake m...
1       forest fire near la ronge sask  canada no_keyword
2       all residents asked to 'shelter in place' are ...
3              people receive  wildfires evacuation or...
4       just got sent this photo from ruby  alaska as ...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609     aria ahrary  thetawniest the out of control w...
7610    m            utc   km s of volcano hawaii  htt...
7611    police investigating after an e bike collided ...
7612    the latest  more homes razed by northern calif...
Name: X, Length: 7613, dtype: object

In [15]:
vectorizer = TfidfVectorizer(stop_words=STOPWORDS)

In [20]:
type(df_train['X'])

pandas.core.series.Series

In [21]:
vectorizer.fit_transform(df_train['X'])

c:\Users\Joaquin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


<7613x21997 sparse matrix of type '<class 'numpy.float64'>'
	with 80886 stored elements in Compressed Sparse Row format>

In [22]:
feature_names = vectorizer.get_feature_names() 


c:\Users\Joaquin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
feature_names[:100]

['20accident',
 '20bag',
 '20bagging',
 '20bags',
 '20bang',
 '20bomb',
 '20bomber',
 '20bombing',
 '20buildings',
 '20burning',
 '20collapse',
 '20disaster',
 '20emergency',
 '20failure',
 '20fall',
 '20fire',
 '20fires',
 '20murder',
 '20murderer',
 '20on',
 '20plan',
 '20reactor',
 '20responders',
 '20services',
 '20spill',
 '20storm',
 '20truck',
 '20up',
 '20wave',
 '20zone',
 'aa',
 'aaaa',
 'aaaaaaallll',
 'aaaaaand',
 'aaarrrgghhh',
 'aace',
 'aag',
 'aal',
 'aamir',
 'aan',
 'aannnnd',
 'aar',
 'aaronthefm',
 'aashiqui',
 'aatt',
 'aauizggc',
 'aawzxykles',
 'ab',
 'aba',
 'abandon',
 'abandoned',
 'abandonedpics',
 'abandoning',
 'abbandoned',
 'abbott',
 'abbruchsimulator',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcnews',
 'abcnorio',
 'abe',
 'aberdeen',
 'aberdeenfanpage',
 'aberdeenfc',
 'aberystwyth',
 'abgct',
 'abgfglh',
 'abha',
 'abia',
 'ability',
 'abject',
 'ablaze',
 'able',
 'ableg',
 'ablz',
 'abninfvet',
 'abnzqwlig',
 'aboard',
 'abolxmh',
 'abomb',
 'abom

In [31]:
X_train,X_test,y_train,y_test = train_test_split(df_train.X,df_train.target,test_size=0.3)

In [32]:
X_train_Tfidf = vectorizer.transform(X_train)
X_test_Tfidf = vectorizer.transform(X_test)

In [37]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_Tfidf,y_train)

SVC(gamma='auto', kernel='linear')

In [38]:
prediction_SVM = SVM.predict(X_test_Tfidf)

In [40]:
accuracy_score(y_test,prediction_SVM)

0.8016637478108581

In [41]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [43]:
df_test['clean-text'] = df_test.text.apply(lambda x: clean_text(x))

In [44]:
df_test['clean-text']

0                      just happened a terrible car crash
1       heard about  earthquake is different cities  s...
2       there is a forest fire at spot pond  geese are...
3                apocalypse lighting   spokane  wildfires
4           typhoon soudelor kills    in china and taiwan
                              ...                        
3258    earthquake safety los angeles     safety faste...
3259    storm in ri worse than last hurricane  my city...
3260    green line derailment in chicago http   t co u...
3261    meg issues hazardous weather outlook  hwo  htt...
3262     cityofcalgary has activated its municipal eme...
Name: clean-text, Length: 3263, dtype: object

In [45]:
df_test_Tidf = vectorizer.transform(df_test['clean-text'])

In [46]:
pred_SVM = SVM.predict(df_test_Tidf)

In [49]:
new_dict = {}
new_dict['id'] = df_test.id
new_dict['target'] = pred_SVM

In [50]:
result_df = pd.DataFrame(new_dict)

In [52]:
result_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [54]:
result_df.to_csv('./result_svm.csv',index=False)